In [2]:
import clickhouse_connect

if __name__ == '__main__':
    client = clickhouse_connect.get_client(
        host='wajbbmzq6a.europe-west4.gcp.clickhouse.cloud',
        user='default',
        password='3phq~SvlfTr39',
        secure=True
    )
    print("Result:", client.query("SELECT 1").result_set[0][0])

Result: 1
